In [1]:
%matplotlib notebook
import os
import sys
import cv2
import datetime
import numpy as np
import matplotlib as plt
from darkflow.net.build import TFNet
import track.sort.sort as srt
from track.eliminateoverlaps import *
import tensorflow as tf

In [35]:
%run flow --model 'cfg/yolo.2.0.cfg' --load 'cfg/yolo.2.0.weights' --savepb

Parsing ./cfg/yolo.2.0.cfg
Parsing cfg/yolo.2.0.cfg
Loading cfg/yolo.2.0.weights ...
Successfully identified 269862452 bytes
Finished in 0.014054059982299805s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 416, 416, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 128)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      

NameError: name 'exit' is not defined

In [38]:
# directory with images from video
#working_directory = '/home/analyticsuser/data/Husband Surprises Wife by Filling House With Puppies!'
# working_directory = '/home/analyticsuser/data/Nest Cam - Owner catches burglars in act on her phone'
# working_directory = '/data01/Detection/JPG'

# # only images
# files_in_directory = sorted([os.path.join(working_directory, file_name)
#                       for file_name in os.listdir(working_directory)
#                       if os.path.splitext(file_name)[1].lower() in ('.jpg', '.jpeg')])

# colors for different bboxes for visualization
colors = [(255,0,0), (0,255,0),(0,0,255),(255,255,0),(0,255,255),(255,0,255),(192,192,192),(128,128,128),
          (128,0,0),(128,128,0),(0,128,0),(128,0,128),(0,128,128),(0,0,128),(0,0,0)]

# configuration for YOLO
options = {'model': '/home/analyticsuser/darkflow/cfg/yolo.2.0.cfg', 
           'load': '/home/analyticsuser/darkflow/cfg/yolo.2.0.weights', 'threshold': 0.1}

# options = {'model': '/home/analyticsuser/darkflow/cfg/yolo-voc.2.0.cfg', 
#            'load': '/home/analyticsuser/darkflow/cfg/yolo-voc.2.0.weights', 'threshold': 0.1}

tfnet = TFNet(options)
#tfnet.savepb()

# initialize SORT tracker:
# - 10 consecutive frames with no detection to terminate an existing track
# - 1 consecutive frame (i.e. after an initial frame) to activate the new track (before that will not produce any output)
# - 5 frames to extrapolate an active track (i.e. show predicted output eventhough no detection present for this track)
# - 0.8 unassigned detection over track bbox overlap (intersection over union) threshold to eliminate the detection
# - 0.94 unassigned detection over another stronger unassigned detection overlap (intersection over self bbox area) threshold to eliminate the detection
# - 0.8 unassigned detection over another stronger unassigned detection overlap (intersection over larger bbox area) threshold to eliminate the detection
# - 2.0 defines "stronger" detection disregarding the size -- if the score of the unassigned detection overlapping 
#       another unassigned detection is 2.0-times higher than the score of the other detection, it is stronger, otherwise 
#       larger size decides

#mot_tracker = srt.Sort(10,0,10,0.55,0.90,0.6,2.0)

Parsing ./cfg/yolo.2.0.cfg
Parsing /home/analyticsuser/darkflow/cfg/yolo.2.0.cfg
Loading /home/analyticsuser/darkflow/cfg/yolo.2.0.weights ...
Successfully identified 269862452 bytes
Finished in 0.013426065444946289s


# darkflow/net/build.py - saving the graph  tf.writer_graph

In [40]:
darknet_pb = tfnet.to_darknet()
flags_pb = tfnet.FLAGS
flags_pb.verbalise = False

flags_pb.train = False
# rebuild another tfnet. all const.
tfnet_pb = TFNet(flags_pb, darknet_pb)		
tfnet_pb.sess = tf.Session(graph = tfnet_pb.graph)

# tfnet_pb.predict() # uncomment for unit testing
name = 'graph-{}.pb'.format(tfnet.meta['name'])
graph_def = tfnet_pb.sess.graph#_def


#tf.train.write_graph(graph_def,'/home/analyticsuser/','yolo_2_0.pb',False)
#tf.summary.FileWriter('/home/analyticsuser/yolo_graph_log/3', graph_def)

In [41]:
# run the session
graph_def = tfnet_pb.sess.graph_def
with tf.gfile.GFile('/home/analyticsuser/yolo_graph_def.pb', "wb") as f:
    f.write(graph_def.SerializeToString())

In [42]:
# # print operations
op = tfnet_pb.sess.graph.get_operations()
for m in op:
    print(m.values())

# a = tfnet.sess
# # print variables    
# for v in tf.global_variables() :
#     print(v)
    
# for i in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES) :
#     print(i)

(<tf.Tensor 'input:0' shape=(?, 416, 416, 3) dtype=float32>,)
(<tf.Tensor 'Pad/paddings:0' shape=(4, 2) dtype=int32>,)
(<tf.Tensor 'Pad:0' shape=(?, 418, 418, 3) dtype=float32>,)
(<tf.Tensor '0-convolutional/filter:0' shape=(3, 3, 3, 32) dtype=float32>,)
(<tf.Tensor '0-convolutional:0' shape=(?, 416, 416, 32) dtype=float32>,)
(<tf.Tensor 'sub/y:0' shape=(32,) dtype=float32>,)
(<tf.Tensor 'sub:0' shape=(?, 416, 416, 32) dtype=float32>,)
(<tf.Tensor 'truediv/y:0' shape=(32,) dtype=float32>,)
(<tf.Tensor 'truediv:0' shape=(?, 416, 416, 32) dtype=float32>,)
(<tf.Tensor 'mul/y:0' shape=(32,) dtype=float32>,)
(<tf.Tensor 'mul:0' shape=(?, 416, 416, 32) dtype=float32>,)
(<tf.Tensor 'BiasAdd/bias:0' shape=(32,) dtype=float32>,)
(<tf.Tensor 'BiasAdd:0' shape=(?, 416, 416, 32) dtype=float32>,)
(<tf.Tensor 'mul_1/x:0' shape=() dtype=float32>,)
(<tf.Tensor 'mul_1:0' shape=(?, 416, 416, 32) dtype=float32>,)
(<tf.Tensor '1-leaky:0' shape=(?, 416, 416, 32) dtype=float32>,)
(<tf.Tensor '2-maxpool:0' s

# Loading the YOLO graph tf.import_graph

In [36]:
from tensorflow.python.platform import gfile

with gfile.FastGFile("/home/analyticsuser/tensorflow_inception_graph.pb",'rb') as f:
#with gfile.FastGFile("/home/analyticsuser/yolo_2.pb",'rb') as f:
#with gfile.FastGFile("/home/analyticsuser/test.pb",'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.import_graph_def(graph_def, name='')

#tf.summary.FileWriter('/home/analyticsuser/inception_graph_log/1', graph_def)
#tf.summary.FileWriter('/home/analyticsuser/yolo_graph_log/3', graph_def)
#tf.summary.FileWriter('/home/analyticsuser/yolo_graph_log/4', graph_def)

# List the operations in the graph
https://stackoverflow.com/questions/35336648/list-of-tensor-names-in-graph-in-tensorflow

In [37]:
#print(tf.__version__)

# sess = tf.Session()
# op = sess.graph.get_operations()
# #print([m.values() for m in op])
# for m in op:
#     print(m.values())

# to see the names of all operation names in the graph, no need to open session
for i in tf.get_default_graph().get_operations():
    print(i.name)

DecodeJpeg/contents
DecodeJpeg
Cast
ExpandDims/dim
ExpandDims
ResizeBilinear/size
ResizeBilinear
Sub/y
Sub
Mul/y
Mul
conv/conv2d_params
conv/Conv2D
conv/batchnorm/beta
conv/batchnorm/gamma
conv/batchnorm/moving_mean
conv/batchnorm/moving_variance
conv/batchnorm
conv/CheckNumerics
conv/control_dependency
conv
conv_1/conv2d_params
conv_1/Conv2D
conv_1/batchnorm/beta
conv_1/batchnorm/gamma
conv_1/batchnorm/moving_mean
conv_1/batchnorm/moving_variance
conv_1/batchnorm
conv_1/CheckNumerics
conv_1/control_dependency
conv_1
conv_2/conv2d_params
conv_2/Conv2D
conv_2/batchnorm/beta
conv_2/batchnorm/gamma
conv_2/batchnorm/moving_mean
conv_2/batchnorm/moving_variance
conv_2/batchnorm
conv_2/CheckNumerics
conv_2/control_dependency
conv_2
pool/CheckNumerics
pool/control_dependency
pool
conv_3/conv2d_params
conv_3/Conv2D
conv_3/batchnorm/beta
conv_3/batchnorm/gamma
conv_3/batchnorm/moving_mean
conv_3/batchnorm/moving_variance
conv_3/batchnorm
conv_3/CheckNumerics
conv_3/control_dependency
conv_3
con

# WORKING SOLUTION (model can be loaded to SNPE) - A frozen *.pb graph can be loaded using tf.import_graph_def() and then saved by tf.gFile.GFile 
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/tools/freeze_graph_test.py#L56

able to read inception_v3, yolo_v2, output_graph.pb (simple 2 layer graph) ...

In [ ]:
from tensorflow.python.framework import ops
from tensorflow.core.framework import graph_pb2
from tensorflow.python.framework import importer

output_graph_path = "/home/analyticsuser/tensorflow_inception_graph.pb"
#output_graph_path = "/home/analyticsuser/inception_jana.pb"
#output_graph_path = "/home/analyticsuser/yolo_2_0_tfnet.pb"
#output_graph_path = "/home/analyticsuser/yolo_2.pb"
#output_graph_path = "/home/analyticsuser/test.pb"
#output_graph_path = "/home/analyticsuser/output_graph.pb"


with ops.Graph().as_default():
    output_graph_def = graph_pb2.GraphDef()
    with open(output_graph_path, "rb") as f:
        output_graph_def.ParseFromString(f.read())
        _ = importer.import_graph_def(output_graph_def, name="")
 


#tf.summary.FileWriter('/home/analyticsuser/yolo_graph_log/4', output_graph_def)
#tf.summary.FileWriter('/home/analyticsuser/inception_graph_log/5', output_graph_def)
#----------------------
# save the graph again
#----------------------
#tf.train.write_graph(output_graph_def, "/home/analyticsuser/", "test.pb", False)

#Save the imported graph using gfile
#-----------------------------------
with tf.gfile.GFile('/home/analyticsuser/yolo_test_graph.pb', "wb") as f:
    f.write(output_graph_def.SerializeToString())

In [ ]:

sess = tf.Session(graph = output_graph_def.graph)

tf.train.write_graph(sess.graph_def,'/home/analyticsuser/','inception_jana.pb',False)

# Save the  Meta Graph and freeze it

In [ ]:
import os
from tensorflow.core.framework import graph_pb2
from tensorflow.core.protobuf import saver_pb2
from tensorflow.python.client import session
from tensorflow.python.framework import graph_io
from tensorflow.python.framework import importer
from tensorflow.python.framework import ops
from tensorflow.python.framework import test_util
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import variables
from tensorflow.python.platform import test
from tensorflow.python.tools import freeze_graph
from tensorflow.python.training import saver as saver_lib

temp_dir = '/home/analyticsuser/'


checkpoint_prefix = os.path.join(temp_dir, "saved_checkpoint")
checkpoint_state_name = "checkpoint_state"
input_graph_name = "input_graph.pb"
output_graph_name = "output_graph.pb"

# We'll create an input graph that has a single variable containing 1.0,
# and that then multiplies it by 2.
with ops.Graph().as_default():
    variable_node = variables.Variable(1.0, name="variable_node")
    output_node = math_ops.multiply(variable_node, 2.0, name="output_node")
    sess = session.Session()
    init = variables.global_variables_initializer()
    sess.run(init)
    output = sess.run(output_node)
    #tf.assertNear(2.0, output, 0.00001)
    saver_write_version  = saver_pb2.SaverDef.V2
    saver = saver_lib.Saver(write_version=saver_write_version)
    checkpoint_path = saver.save(
      sess,
      checkpoint_prefix,
      global_step=0,
      latest_filename=checkpoint_state_name)
    
    graph_io.write_graph(sess.graph, temp_dir, input_graph_name)
 

# We save out the graph to disk, and then call the const conversion
# routine.
input_graph_path = os.path.join(temp_dir, input_graph_name)
input_saver_def_path = ""
input_binary = False
output_node_names = "output_node"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_graph_path = os.path.join(temp_dir, output_graph_name)
clear_devices = False

freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_graph_path, clear_devices, "")

# Freeze graph - not working
https://github.com/metaflow-ai/blog/blob/master/tf-freeze/freeze.py

In [ ]:
# from tensorflow.python.framework import graph_util

# # We start a session and restore the graph weights
# with tf.Session(graph = tfnet_pb.graph) as sess:
    
#     # We use a built-in TF helper to export variables to constants
#     output_graph_def = graph_util.convert_variables_to_constants(
#         sess, # The session is used to retrieve the weights
#         graph_def, # The graph_def is used to retrieve the nodes 
#         output_node_names.split(",") # The output node names are used to select the usefull nodes
#     ) 

#     with tf.gfile.GFile('test_graph.pb', "wb") as f:
#         f.write(output_graph_def.SerializeToString())
#     print("%d ops in the final graph." % len(output_graph_def.node))

In [ ]:
# darknet_pb = tfnet.to_darknet()
# flags_pb = tfnet.FLAGS
# flags_pb.verbalise = False

# flags_pb.train = False
# # rebuild another tfnet. all const.
# tfnet_pb = TFNet(flags_pb, darknet_pb)		
# tfnet_pb.sess = tf.Session(graph = tfnet_pb.graph)
# # tfnet_pb.predict() # uncomment for unit testing
# name = 'graph-{}.pb'.format(tfnet.meta['name'])
# graph_def = tfnet_pb.sess.graph#_def

# with ops.Graph().as_default():
#     init = variables.global_variables_initializer()
#     tfnet_pb.sess.run(init)    

#     saver_write_version  = saver_pb2.SaverDef.V2
#     saver = saver_lib.Saver(write_version=saver_write_version)
#     checkpoint_path = saver.save(
#       tfnet_pb.sess,
#       checkpoint_prefix,
#       global_step=0,
#       latest_filename=checkpoint_state_name)
#     graph_io.write_graph(graph_def, temp_dir, input_graph_name)
    
# freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
#                           input_binary, checkpoint_path, output_node_names,
#                           restore_op_name, filename_tensor_name,
#                           output_graph_path, clear_devices, "")    

# useless example

In [ ]:
	def savepb_freeze(self):
		"""
		Load a standalone const graph def and then freezit
		"""
		temp_dir = '/home/analyticsuser/'
		input_graph_name = "yolo.2.0.pb"
		output_graph_name = "yolo.2.0_freeze.pb"
		input_graph_path = os.path.join(temp_dir, input_graph_name)
		input_saver_def_path = ""
		input_binary = False
		output_node_names = "output_node"
		restore_op_name = "save/restore_all"
		filename_tensor_name = "save/Const:0"
		output_graph_path = os.path.join(temp_dir, output_graph_name)
		clear_devices = False
		
		saver = saver_lib.Saver(write_version=saver_write_version)
		checkpoint_path = saver.save(
			sess,
			checkpoint_prefix,
			global_step=0,
			latest_filename=checkpoint_state_name)        
		freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                                  input_binary, checkpoint_path, output_node_names,
                                  restore_op_name, filename_tensor_name,
                                  output_graph_path, clear_devices, "")   

In [ ]:
tfnet.savepb()

In [ ]:
# detections_dir = '/data01/Detections'
# detections_dir_csv = '/data01/Detections/CSV'
# if not os.path.exists(detections_dir):
#     os.mkdir(detections_dir)
    
# # init a dataframe with raw detections
# detections_info_file = 'Detections_' + datetime.date.strftime(datetime.datetime.now(),'%Y-%m-%d %H-%M') + '.csv'
# with open(os.path.join(detections_dir_csv,  detections_info_file), 'w') as output:
#     output.write('Date,Timestamp,Filename,TopLeftX,TopLeftY,BottomRightX,BottomRightY,Confidence\n')

In [ ]:
plt.ion()

# init Camrea reader & matplotlib displaying
# cap = cv2.VideoCapture('/data01/Detection/JPG/2017-05-16 23-12-40.059.jpg')
# print(cap)
# ret, imgcv = cap.read()

imgcv = cv2.imread('/data01/Detections/JPG/2017-05-16 23-12-40.059.jpg')
imgcv_to_display = cv2.cvtColor(imgcv, cv2.COLOR_BGR2RGB)
# pltimage = np.concatenate((imgcv_to_display, imgcv_to_display), axis=1)
fig=plt.figure(figsize=(16,12), dpi= 80, facecolor='w', edgecolor='k')
# detections_preview = plt.imshow(pltimage)
detections_preview = plt.imshow(imgcv)
try:
    plt.pause(0.05)
except Exception:
    pass

# displaying all
# allres = []
frmcnt=0
while(True):
    sys.stdout.write("\r" + str(frmcnt))
    sys.stdout.flush()
    frmcnt+=1
    
    #ret, imgcv = cap.read()
    imgcv = cv2.imread('/data01/Detections/JPG/2017-05-16 23-12-40.059.jpg')
    if type(imgcv) == type(None):
        print("!!! Couldn't read frame!")
        break    

    img1 = imgcv.copy()
    result = tfnet.return_predict(imgcv)
#     for idx, detection in enumerate(result):
#         if detection['label'] == 'person':
# #         if detection['label'] == 'person' or detection['label'] == 'dog' or detection['label'] == 'cat':
#             cv2.rectangle(imgcv, (detection['topleft']['x'], detection['topleft']['y']), 
#                           (detection['bottomright']['x'], detection['bottomright']['y']), 
#                           colors[idx % len(colors)], 10)

#             cv2.rectangle(imgcv, (detection['topleft']['x'], detection['topleft']['y'] - 20), 
#                           (detection['bottomright']['x'], detection['topleft']['y']), (125, 125, 125), -1)

#             cv2.putText(imgcv, detection['label'] + ' : %.2f' % detection['confidence'], 
#                         (detection['topleft']['x'] + 5, detection['topleft']['y'] - 7), 
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            
            
       
    
    # continue with tracking
    # print('result: ', result) 
#     fres = list(filter(lambda x: x['label'] == 'person' or x['label'] == 'dog' or x['label'] == 'cat',result))
    fres = list(filter(lambda x: x['label'] == 'person',result))
    dets0 = list(map( lambda x: [float(x['topleft']['x']),float(x['topleft']['y']),
                                 float(x['bottomright']['x']),float(x['bottomright']['y']),
                                 x['confidence'],x['label']], fres))
    if len(dets0):
        current_time = datetime.datetime.now()
        #file_name = str(current_time) + '.jpg'
        file_name = datetime.date.strftime(current_time, '%Y-%m-%d %H-%M-%S.%f')[:-3] + '.jpg'
        #cv2.imwrite(os.path.join(detections_dir, file_name), img1)
        
    # eliminate overlaps
    eldets=eliminate1(dets0,0.80,0.6,2.0)
    for idx, bbo_f in enumerate(eldets):
        # print('bbo_f: ', bbo_f )
        bbo = list(map(lambda x: int(round(float(x))), bbo_f[0:4]))
        obsc = bbo_f[4]
        label = bbo_f[5]
        cv2.rectangle(img1, (bbo[0], bbo[1]), (bbo[2], bbo[3]), colors[idx%len(colors)], 10)
        cv2.rectangle(img1, (bbo[0], bbo[1]-30), (bbo[2], bbo[1]), colors[idx%len(colors)], -1)
        cv2.putText(img1, label+' %0.2f'%obsc, (bbo[0]+5, bbo[1]-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,0), 2)
        
#         print(len(bbo))
#         with open(os.path.join(detections_dir_csv, detections_info_file), 'a') as output_file:
# #             'Date,Timestamp,ID,Filename,TopLeftX,TopLeftY,BottomRightX,BottomRightY,Confidence\n'
#             output_file.write(f"{datetime.date.strftime(current_time,'%Y-%m-%d')},"
#                               f"{'2000-01-01 ' + datetime.date.strftime(current_time, '%H:%M:%S.%f')[:-3]},"
#                               f"{file_name},{bbo[0]},{bbo[1]},{bbo[2]},{bbo[3]},{obsc}\n")
    
    # print('dets0: ', dets0) 
    # allres += [','.join(list(map(lambda x: str(x),dets0)))]   # save the results
#     dets = np.asarray( dets0 )
#     objs,termtracks = mot_tracker.update(dets)
    # print('objs: ', objs) 
#     for bbo_f in objs:
#         # print('bbo_f: ', bbo_f )
#         bbo = list(map(lambda x: int(round(float(x))), bbo_f[0:4]))
#         obid = int(bbo_f[5])
#         obsc = bbo_f[4]
#         cv2.rectangle(img1, (bbo[0], bbo[1]), (bbo[2], bbo[3]), colors[obid%len(colors)], 10)
#         cv2.rectangle(img1, (bbo[0], bbo[1]-40), (bbo[2], bbo[1]), (125, 125, 125), -1)
#         cv2.putText(img1, 'id=%03d'%obid + ' @%0.2f'%obsc, (bbo[0]+5, bbo[1]-7), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)

#     if len(dets0):
#         cv2.imwrite(os.path.join(detections_dir, 'JPG', file_name), img1) # save image with detections
    
    imgcv_to_display = cv2.cvtColor(imgcv, cv2.COLOR_BGR2RGB)
    
    imgcv_to_display2 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
            
#     pltimage = np.concatenate((imgcv_to_display, imgcv_to_display2), axis=1)
#     detections_preview.set_data(pltimage)
    detections_preview.set_data(imgcv_to_display2)

#     plt.draw()
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
#         plt.pause(0.0002)
    except Exception:
        pass

#content = '\n'.join(allres)
#with open('yolodet.txt', 'w') as fro:
#    fro.write(content)


In [ ]:
class AlexNetModel():
    # ...
    def build_model(self, x):
        assert x.get_shape().as_list() == [224, 224, 3]
        self.conv1 = conv_2d(x, 96, 11, strides=4, activation='relu')
        self.pool1 = max_pool_2d(self.conv1, 3, strides=2)
        self.conv2 = conv_2d(self.pool1, 256, 5, activation='relu')
        self.pool2 = max_pool_2d(self.conv2, 3, strides=2)
        self.conv3 = conv_2d(self.pool2, 384, 3, activation='relu')
        self.conv4 = conv_2d(self.conv3, 384, 3, activation='relu')
        self.conv5 = conv_2d(self.conv4, 256, 3, activation='relu')
        self.pool5 = max_pool_2d(self.conv5, 3, strides=2)
        self.fc6 = fully_connected(self.pool5, 4096, activation='relu')
        self.fc7 = fully_connected(self.fc6, 4096, activation='relu')
        self.output = fully_connected(self.fc7, 1000, activation='softmax')
        return self.output
model = AlexNetModel()
output = model.build_model(images)